In [187]:
import requests
from bs4 import BeautifulSoup  
import re
import pandas as pd

In [188]:
url = "https://www.mhlw.go.jp/stf/houdou/houdou_list_202001.html"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

page = requests.get(url)

page.encoding = "utf-8"
html=page.text
soup = BeautifulSoup(html, 'lxml')

In [189]:
linklist = []
linkdict = {}
for x in soup.find_all('a',href = re.compile('newpage')):
#     print(x)
    string = x.get_text()
    string = re.sub(r'[NEW\s ,]*','',string)
    if "例目" in string:
#         print(string)
        link = x.get('href')
        if link:
            linkdict[string] =  "https://www.mhlw.go.jp" + link

In [190]:
def input_mode(status,string):
    label = ''
    out1 = ''
    x= 5
    if status == 0:
        out1 = 'int' + string
    elif status%5 == 1:
        out1 = 'age' + string[6:]    
    elif status%5 == 2:
        out1 = 'sex' + string[6:]    
    elif status%5 == 3:
        out1 = 'loc' + string[7:]    
    elif status%5 == 4:
        out1 = 'spt' + string
    elif status%5 == 0 and status != 0:
        if string[0] == '無':
            out1 = 'int' + string   
        else:
            out1 = 'rdl' + string   
        
    return out1

In [227]:
def petient(urls):
    url = urls
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    page = requests.get(url)
    page.encoding = "utf-8"
    html=page.text
    soup = BeautifulSoup(html, 'lxml')
    linklist = []
    linkdict = {}
    text = ''
    for x in soup.find_all('div','m-grid__col1'):
        string = x.get_text()
        string = re.sub(r'[　]*','',string)
        if "居住" in string:
            text = string
    k = text.split('\n')
    count = 0
    new = []
    for i in range(len(k)):
        if len(k[i]) > 3:
            count = count + 1
            str1 = k[i]
            if str1[0] == "◆":
                break
            else:
                new.append(k[i])
        else:
            count = count + 1
#     final = []
#     status = 0
#     for i in range(len(new)):
#         if new[i][0] == '（' and new[i][1].isdigit() == True:
#             status = status + 1
#             labeltext = input_mode(status,new[i])
#             final.append(labeltext)
#         else:
#             labeltext = input_mode(status,new[i])
#             final.append(labeltext)
    final = []
    status = 0
    for i in range(len(new)):
        if new[i][0] == '（' :
            status = status_check(new[i])
            labeltext = input_mode(status,new[i])
            final.append(labeltext)
        else:
            labeltext = input_mode(status,new[i])
            final.append(labeltext)

    return final
    

In [253]:
def status_check(string):
    if '）年代' in string[2:]:
        status = 1
    elif '）性別' in string[2:]:
        status= 2
    elif '）居住' in string[2:]:
        status= 3
    elif '）症状' in string[2:]:
        status= 4
    elif '行動歴' in string[2:]:
        status= 5
    else :
        status = 0
    return status

In [259]:
def number_check(age_count, i):
    if age_count > 1:
        i = i + 1
    return i

In [262]:
i = 0
df =pd.DataFrame(columns=('num','age','sex','loc','spt','rdl','title','weblink'))
for key in linkdict:
    petient_url = linkdict[key]
    describe = petient(petient_url)
    print(key)
    print(linkdict[key])
    symptom = []
    roadline = []
    df.loc[i,'title'] = key
    df.loc[i,'weblink'] = linkdict[key]
    df.loc[i,'num'] = re.findall(r'\d+', key)
    age_count = 0
    for phase in describe:
        phase = phase.strip()
        if phase[:3]== 'age':
            age_count = age_count + 1
            past = i
            i = number_check(age_count,i)
            if past!=i:
                df.loc[i-1,'spt'] = symptom
                df.loc[i-1,'rdl'] = roadline
                symptom = []
                roadline = []
                age_count = 1
            df.loc[i,'age'] = phase[3:]
        elif phase[:3]== 'sex' : 
            df.loc[i,'sex'] = phase[3:]
        elif phase[:3]== 'loc' :
            df.loc[i,'loc'] = phase[3:]
        elif phase[:3]== 'spt':
            symptom.append(phase[3:])
        elif phase[:3]== 'rdl':
            roadline.append(phase[3:])
        print(phase)
    df.loc[i,'spt'] = symptom
    df.loc[i,'rdl'] = roadline
    i = i+1 
    print('============================')

新型コロナウイルスに関連した患者（14、15例目）及び無症状病原体保有者（＊）の発生について
https://www.mhlw.go.jp/stf/newpage_09278.html
int昨日（1月31日）、武漢市からのチャーター便（第３便）により帰国した邦人について、本日15時、国立感染症研究所において新型コロナウイルスに係る検査を実施したところ、今般の新型コロナウイルスに関連した感染症の２症例の報告がありました。２例のうち１例（14例目の患者）は国立国際医療研究センター（NCGM）における精査で症状が認められた15人のうちの１人です。もう１例は無症状病原体保有者です。
intまた1月29日、武漢市からのチャーター便（第１便）により帰国して医療機関に入院した邦人12人のうち新型コロナウイルスに係る検査で陰性と報告のあった１人について、ウイルス性肺炎である可能性が高いと医療機関が判断し、本日15時、国立感染症研究所において新型コロナウイルスに係る追加検査を実施したところ、陽性との報告がありました（15例目の患者）。
intこれらの方は武漢市からのチャーター便により帰国された邦人でPCR検査の結果、新型コロナウイルスが検出されました。本件について、濃厚接触者の把握を含めた積極的疫学調査を確実に行ってまいります。
int患者14例目
age40代
sex男性
loc調査中
spt（４）症状、経過：
spt１月31日 第３便で帰国。国立国際医療研究センターにおける診察時に咳と38℃台の発熱を認めたため検査を行ったところ、陽性。
rdl（５）行動歴：
rdl本人からの報告によれば、武漢市の海鮮市場（華南海鮮城）には立ち寄っていない。
rdl中国において、肺炎患者との明確な接触は確認できていない。
rdl患者15例目
age40代
sex男性
loc中華人民共和国
spt（４）症状、経過：
spt１月26日より咳が出現。１月29日の帰国時に37℃台の発熱と咳を認め、入院となった。１回目の検査は陰性。その後、喀痰を用いて追加検査を行ったところ、陽性との結果を得た。
spt症状は改善傾向であり、２月１日現在、咳は少々残るものの、解熱している。
rdl（５）行動歴：
rdl本人からの報告によれば、武漢市の海鮮市場（華南海鮮城）には立ち寄っていない。
rdl中国において、肺炎患者

新型コロナウイルスに関連した肺炎の患者の発生について（2例目）
https://www.mhlw.go.jp/stf/newpage_09079.html
int本日（1月24日）１時頃に、国立感染症研究所より、今般の新型コロナウイルスに関連した感染症の症例の報告がありました。
intこの患者は、中華人民共和国湖北省武漢市在住の旅行者であり、１月20日にご本人が医療機関を受診した際に、武漢市の滞在歴の申告があったとして、報告がされたものです。
int新型コロナウイルスに関連した感染症の患者の発生が国内で確認されたのは２例目です。
int本件について、濃厚接触者の把握を含めた積極的疫学調査を確実に行ってまいります。
age40代
sex男性
loc中華人民共和国（湖北省武漢市）
spt（４）症状、経過：
spt１月14日から発熱あり。
spt１月15、17日に医療機関を受診し肺炎の診断はなく、経過観察
spt１月19日に来日（症状は落ち着いていた）
spt１月20日に医療機関を受診し、肺炎の診断なく、経過観察
spt１月22日発熱、咽頭痛が持続するため、医療機関を受診し、肺炎像を認め、東京都内医療機関に転院し、現在入院中。
rdl（５）行動歴：本人からの報告によれば、武漢市の海鮮市場（華南海鮮城）には立ち寄っていない。
rdl中国において、肺炎患者との明確な接触は確認できていない。
rdl本人は、同行者と別の部屋に宿泊しており、ほぼ常に部屋に滞在していた。
rdlなお、移動時にはマスクを着用していたとのこと。
新型コロナウイルスに関連した肺炎の患者の発生について（1例目）
https://www.mhlw.go.jp/stf/newpage_08906.html
int１月14日、神奈川県内の医療機関から管轄の保健所に対して、中華人民共和国湖北省武漢市の滞在歴がある肺炎の患者が報告されました。この方については、１月６日にご本人が医療機関を受診した際に、武漢市の滞在歴の申告があり、その後、原因が明らかでない肺炎等の患者に係る、国立感染症研究所での検査制度（疑似症サーベイランス）に基づき報告されたものです。
int当該患者の検体を国立感染症研究所（村山庁舎）で検査したところ、昨日（１月15日）20時45分頃に新型コロナウイルス陽性の結果が得られました。新型コロナウイルスに

In [263]:
df

,num,age,sex,loc,spt,rdl,title,weblink
0,"[14, 15]",40代,男性,調査中,"[（４）症状、経過：, １月31日 第３便で帰国。国立国際医療研究センターにおける診察時に咳...","[（５）行動歴：, 本人からの報告によれば、武漢市の海鮮市場（華南海鮮城）には立ち寄っていな...",新型コロナウイルスに関連した患者（14、15例目）及び無症状病原体保有者（＊）の発生について,https://www.mhlw.go.jp/stf/newpage_09278.html
1,NaN,40代,男性,中華人民共和国,"[（４）症状、経過：, １月26日より咳が出現。１月29日の帰国時に37℃台の発熱と咳を認め...","[（５）行動歴：, 本人からの報告によれば、武漢市の海鮮市場（華南海鮮城）には立ち寄っていな...",NaN,NaN
2,NaN,30代,男性,調査中,"[（４）症状、経過：, １月31日 第３便で帰国。症状はなく、念のため検査で陽性。]","[（５）行動歴：, 本人からの報告によれば、武漢市の海鮮市場（華南海鮮城）には立ち寄っていな...",NaN,NaN
3,[13],20代,女性,千葉県,"[（４）症状、経過：, 1月20日頃から咳、鼻水症状あり。, 1月29日鼻水が続くため、医療...","[（５）行動歴：, 1月16日 東京から大阪へ飛行機で移動。, 1月17-22日にバスガイド...",新型コロナウイルスに関連した肺炎の患者の発生について（13例目）,https://www.mhlw.go.jp/stf/newpage_09271.html
4,[11],30代,女性,中華人民共和国（湖南省）,"[（４）症状、経過：, １月19日に武漢市に滞在。, １月20日に入国。入国時は症状なし。2...","[（５）行動歴：, 本人からの報告によれば、武漢市の海鮮市場（華南海鮮城）には立ち寄っていな...",新型コロナウイルスに関連した肺炎の患者の発生について（11例目）,https://www.mhlw.go.jp/stf/newpage_09237.html
5,[12],20代,女性,京都府,"[（４）症状、経過：, １月16-22日に武漢市に滞在。, １月22日午後に入国。入国時点で...","[（５）行動歴：, １月16-22日に武漢市に滞在。この間、１月17日または18日に咳症状の...",新型コロナウイルスに関連した肺炎の患者の発生について（12例目）,https://www.mhlw.go.jp/stf/newpage_09239.html
6,[10],50代,男性,三重県,"[（４）症状、経過：, １月13日、武漢市から帰国。その際は、症状なし。, １月25-26日...","[（５）行動歴：, 12月24日-１月13日まで武漢市に滞在。その他、行動歴は現在確認中。日...",新型コロナウイルスに関連した肺炎の患者の発生について（10例目）,https://www.mhlw.go.jp/stf/newpage_09236.html
7,[９],５０代,男性,中華人民共和国（湖北省武漢市）,"[（４）症状、経過：１月29日 8時半ごろ帰国。, 医療機関に到着時点で、鼻汁および咽頭痛を...",[（５）行動歴：現在確認中],新型コロナウイルスに関連した患者（９例目）及び無症状病原体保有者（＊）の発生について,https://www.mhlw.go.jp/stf/newpage_09205.html
8,NaN,４０代,男性,調査中,[（４）症状、経過：１月29日 8時半ごろ帰国。症状はなく、念のため検査で陽性。],[（５）行動歴：現在確認中],NaN,NaN
9,NaN,５０代,女性,調査中,[（４）症状、経過：１月29日 8時半ごろ帰国。症状はなく、念のため検査で陽性。],[（５）行動歴：現在確認中],NaN,NaN
